In [1]:
import pandas as pd
import gcsfs as gcs
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [3]:
#data = pd.read_csv('s3://zengageprodadminapp/crimes_sample.csv')
data = pd.read_csv('gs://bigdatahw2/crimes_sample.csv')

In [4]:
sc = SparkContext('local','test')
sqlcont = SQLContext(sc)

In [5]:
data_sc = sqlcont.createDataFrame(data)

In [6]:
data_sc.describe()

DataFrame[summary: string, ID: string, Case Number: string, Date: string, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [11]:
data_2007 = data_sc[data_sc['Year']==2007]

In [12]:
data_2007.describe()

DataFrame[summary: string, ID: string, Case Number: string, Date: string, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [27]:
crime_table_data = data_2007.groupBy(['Primary Type']).count()
#crime_table_data.to_sql('crimestable', conn, schema=None, if_exists='fail', index=True, index_label=None, chunksize=None, dtype=None)

In [28]:
crime_table_data.show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|OFFENSE INVOLVING...|   42|
|            STALKING|    7|
|PUBLIC PEACE VIOL...|   34|
|               ARSON|    4|
|            GAMBLING|    7|
|   CRIMINAL TRESPASS|  119|
|             ASSAULT|  225|
|LIQUOR LAW VIOLATION|   11|
| MOTOR VEHICLE THEFT|  186|
|               THEFT|  918|
|             BATTERY|  793|
|             ROBBERY|  142|
| CRIM SEXUAL ASSAULT|   14|
|        PROSTITUTION|   94|
|  DECEPTIVE PRACTICE|  215|
|         SEX OFFENSE|    9|
|     CRIMINAL DAMAGE|  522|
|           NARCOTICS|  732|
|       OTHER OFFENSE|  282|
|          KIDNAPPING|    1|
+--------------------+-----+
only showing top 20 rows



In [29]:
import mysql.connector

In [53]:
mydb = mysql.connector.connect(
  host="35.192.215.142",
  user="root",
  passwd="XXXXXX",
    database="crimedb"
)
mycursor = mydb.cursor()

In [44]:

mycursor.execute("DROP TABLE crimestable")
mycursor.execute("CREATE TABLE crimestable (crime VARCHAR(255), count int)")

In [45]:
crime_table_data.columns

['Primary Type', 'count']

In [48]:
for crime in crime_table_data.rdd.collect():
    sql = "INSERT INTO crimestable (crime, count) VALUES " + "(" +  crime['Primary Type'] + "," + str(crime['count']) + ")"
    print (sql)


INSERT INTO crimestable (crime, count) VALUES (OFFENSE INVOLVING CHILDREN,42)
INSERT INTO crimestable (crime, count) VALUES (STALKING,7)
INSERT INTO crimestable (crime, count) VALUES (PUBLIC PEACE VIOLATION,34)
INSERT INTO crimestable (crime, count) VALUES (ARSON,4)
INSERT INTO crimestable (crime, count) VALUES (GAMBLING,7)
INSERT INTO crimestable (crime, count) VALUES (CRIMINAL TRESPASS,119)
INSERT INTO crimestable (crime, count) VALUES (ASSAULT,225)
INSERT INTO crimestable (crime, count) VALUES (LIQUOR LAW VIOLATION,11)
INSERT INTO crimestable (crime, count) VALUES (MOTOR VEHICLE THEFT,186)
INSERT INTO crimestable (crime, count) VALUES (THEFT,918)
INSERT INTO crimestable (crime, count) VALUES (BATTERY,793)
INSERT INTO crimestable (crime, count) VALUES (ROBBERY,142)
INSERT INTO crimestable (crime, count) VALUES (CRIM SEXUAL ASSAULT,14)
INSERT INTO crimestable (crime, count) VALUES (PROSTITUTION,94)
INSERT INTO crimestable (crime, count) VALUES (DECEPTIVE PRACTICE,215)
INSERT INTO crim

In [58]:
for crime in crime_table_data.rdd.collect():
    sql = "INSERT INTO crimestable (crime, count) VALUES (%s, %s)"
    val = (crime['Primary Type'], crime['count'])
    mycursor.execute(sql, val)

mydb.commit()

In [59]:
mycursor.execute("Select * from crimestable")
myresult = mycursor.fetchall()

In [60]:
for x in myresult:
  print(x)

('OFFENSE INVOLVING CHILDREN', 42)
('STALKING', 7)
('PUBLIC PEACE VIOLATION', 34)
('ARSON', 4)
('GAMBLING', 7)
('CRIMINAL TRESPASS', 119)
('ASSAULT', 225)
('LIQUOR LAW VIOLATION', 11)
('MOTOR VEHICLE THEFT', 186)
('THEFT', 918)
('BATTERY', 793)
('ROBBERY', 142)
('CRIM SEXUAL ASSAULT', 14)
('PROSTITUTION', 94)
('DECEPTIVE PRACTICE', 215)
('SEX OFFENSE', 9)
('CRIMINAL DAMAGE', 522)
('NARCOTICS', 732)
('OTHER OFFENSE', 282)
('KIDNAPPING', 1)
('BURGLARY', 230)
('WEAPONS VIOLATION', 42)
('INTERFERENCE WITH PUBLIC OFFICER', 8)
